In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline


In [ ]:
ROOT = ".."
filename = f"{ROOT}/data/ZB_WZ_dE_rawdescriptor.csv"
df = pd.read_csv(filename)
DESCRIPTOR_NAMES = ['IP_A', 'EA_A', 'EN_A', 'Highest_occ_A', 
                    'Lowest_unocc_A','rs_A', 'rp_A', 'rd_A', 
                    'IP_B', 'EA_B', 'EN_B', 'Highest_occ_B',
                    'Lowest_unocc_B', 'rs_B', 'rp_B', 'rd_B']
TARGET_NAME = "dE"
RANDOM_STATE = 1
Xraw = df.loc[:, DESCRIPTOR_NAMES].values
yraw = df.loc[:, TARGET_NAME].values


In [ ]:
from sklearn.preprocessing import StandardScaler
y = yraw > 0
scaler = StandardScaler()
scaler.fit(Xraw)
X = scaler.transform(Xraw)


In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegressionCV
kf = KFold(5, shuffle=True, random_state=RANDOM_STATE)
cls_cv = LogisticRegressionCV(cv=kf)
cls_cv.fit(X, y)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
cls = LogisticRegression(C=cls_cv.C_[0])
kf = KFold(5, shuffle=True, random_state=RANDOM_STATE)
yp_cv = cross_val_predict(cls, X, y, cv=kf)
print(classification_report(y, yp_cv))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
AVERAGE = 'weighted'
acc = accuracy_score(y, yp_cv)
prec = precision_score(y, yp_cv, average=AVERAGE)
recall = recall_score(y, yp_cv, average=AVERAGE)
f1 = f1_score(y, yp_cv, average=AVERAGE)
print("accuracy", acc, "precision", prec, "recall", recall, "F1", f1)


In [ ]:
from sklearn.metrics import confusion_matrix
index = []
columns = []
for s in cls_cv.classes_:
    index.append("actual ({})".format(s))
    columns.append("predict ({})".format(s))
df_cm_cv = pd.DataFrame(confusion_matrix(y, yp_cv, labels=cls_cv.classes_), 
                        index=index, columns=columns) 
df_cm_cv


In [ ]:
yp = cls_cv.predict(X)
yp_proba = cls_cv.predict_proba(X)


In [ ]:

df_cm = pd.DataFrame(confusion_matrix(y, yp, labels=cls_cv.classes_), index=index, columns=columns)
df_cm
